In [13]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from auxiliary import values as v
from auxiliary.data import imaging
from auxiliary.utils.timer import LoadingBar
from auxiliary.data.dataset_ht import HtDataset, find_specimen

from nuclei_segmentation.quality_control import metrics, visualization

In [3]:
# v.data_path = '/mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/'

pred_paths = v.data_path + 'SegQA/Segmentation/Nuclei/QC_CROP/'
gt_path = (
        v.data_path 
        + 'SegQA/Segmentation/Nuclei/QC_CROP/20190208_E2_nuclei_mask_crop_GT_filtered.nii.gz'
)

gt = imaging.read_image(gt_path, axes='XYZ')

In [4]:
from itertools import permutations, chain
import numpy as np

# Cellpose params
steps_type = ['3D', '2D']
steps_thr = list(np.round(np.linspace(.2, .6, 6), 1))
steps_prob_thr = list(np.round(np.linspace(0, .4, 4), 1))
steps_flow_thr = list(np.round(np.linspace(.2, .6, 4), 1))

# Preprocessing steps
normalization_methods = ['norm_percentile']
pre_steps_top = ['isotropy']
pre_steps_bottom = [
    'bilateral',
    'anisodiff'
]

pre_steps_permutations = list(chain(*[
    permutations(pre_steps_bottom, r)
    for r in range(1, len(pre_steps_bottom) + 1)
])) + [()]

# Postprocessing steps
post_steps_2d = [
    'merge_connected_components',
    'merge_graph',
    'clean_boundaries_opening',
] + [()]

post_steps_3d = [
    'split',
    'clean_boundaries_opening',
    'clean_boundaries_closing',
]

# Generate permutations of post-processing steps for 3D, excluding combinations with both 'clean_boundaries_opening' and 'clean_boundaries_closing'
post_steps_permutations_3d = [
    p for p in chain(*[
        permutations(post_steps_3d, r)
        for r in range(1, len(post_steps_3d) + 1)
    ]) 
    if not ('clean_boundaries_opening' in p and 'clean_boundaries_closing' in p)
] + [()]

pipelines_dict = {}

# Create pipelines for each type (2D/3D)
for step in steps_type:
    # Apply threshold only for 2D
    if step == '2D':
        for thr in steps_thr:
            for flow in steps_flow_thr:
                for norm_method in normalization_methods:
                    for i, pre_steps in enumerate(pre_steps_permutations):
                        for j, post_steps in enumerate(post_steps_2d):
                            pipeline_pre = pre_steps_top + [norm_method] + list(pre_steps)
                            pipeline = pipeline_pre + [post_steps]

                            # Include threshold for 2D pipelines
                            name = f"pipeline_{step}_thr_{thr}_flow_{flow}_{i}_{j}"
                            pipelines_dict[name] = {
                                'pipeline': pipeline,
                                'type': step,
                                'threshold': thr
                            }
    else:  # For 3D pipelines, ignore thresholds
        for norm_method in normalization_methods:
            for prob_t in steps_prob_thr:
                for flow in steps_flow_thr:
                    for i, pre_steps in enumerate(pre_steps_permutations):
                        for j, post_steps in enumerate(post_steps_permutations_3d):
                            pipeline_pre = pre_steps_top + [norm_method] + list(pre_steps)
                            pipeline = pipeline_pre + list(post_steps)

                            # No threshold for 3D pipelines
                            name = f"pipeline_{step}_prob_{prob_t}_flow_{flow}_{i}_{j}"
                            pipelines_dict[name] = {
                                'pipeline': pipeline,
                                'type': step,
                                'prob_threshold': prob_t
                            }

print('Total pipelines:', len(pipelines_dict))


Total pipelines: 1040


In [5]:
pipelines_dict

{'pipeline_3D_prob_0.0_flow_0.2_0_0': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'split'],
  'type': '3D',
  'prob_threshold': 0.0},
 'pipeline_3D_prob_0.0_flow_0.2_0_1': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'clean_boundaries_opening'],
  'type': '3D',
  'prob_threshold': 0.0},
 'pipeline_3D_prob_0.0_flow_0.2_0_2': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'clean_boundaries_closing'],
  'type': '3D',
  'prob_threshold': 0.0},
 'pipeline_3D_prob_0.0_flow_0.2_0_3': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'split',
   'clean_boundaries_opening'],
  'type': '3D',
  'prob_threshold': 0.0},
 'pipeline_3D_prob_0.0_flow_0.2_0_4': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'split',
   'clean_boundaries_closing'],
  'type': '3D',
  'prob_threshold': 0.0},
 'pipeline_3D_prob_0.0_flow_0.2_0_5': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'clean_bou

In [6]:
bar = LoadingBar(len(pipelines_dict))

results = []
for name, config in pipelines_dict.items():
    print(f"\nPipeline: {name}")
    
    file_name = f'20190208_E2_DAPI_decon_0.5_crop_{name}.nii.gz'
    
    out_path = pred_paths + 'Stats/' + file_name.replace('.nii.gz', '.tif')
    pred = imaging.read_image(pred_paths + file_name, axes='XYZ')
    
    dice, dice_df = metrics.dice_coef(pred, gt, thr_overlap=.6)
    jaccard, _ = metrics.volume_jaccard_index(pred, gt)
    res, stats = metrics.segmentation_stats(pred, gt, thr_overlap=.6)

    results.append({
        'test_name': name,
        'dice': dice,
        'jaccard': jaccard,
        '#cells': stats['total'],
        'correct': stats['correct'],
        'missing': stats['missing'],
        'over_segmented': stats['over_segmented'],
        'under_segmented': stats['under_segmented'],
        'confused': stats['confused']
    })
    
    dice_df.to_csv(pred_paths + 'Dices/' + name + '.csv', index=False)
    visualization.save_comparison(pred, res, out_path)
    bar.update()
    
bar.end()


Pipeline: pipeline_3D_prob_0.0_flow_0.2_0_0
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 117 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 189 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 117 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 189 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 117 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 189 labels...
[                                                  ] 0.10%
Pipeline: pipeline_3D_prob_0.0_flow_0.2_0_1
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 135 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 189 labels...
INFO     [timagetk.components.labelled_image] Searching the bounding-boxes of 135 labels...
INFO     [timagetk.compo

In [7]:
results_df = pd.DataFrame(results)
results_df.to_csv(pred_paths + 'Stats/results.csv', index=False)

In [8]:
results_df

,test_name,dice,jaccard,#cells,correct,missing,over_segmented,under_segmented,confused
0,pipeline_3D_prob_0.0_flow_0.2_0_0,0.564833,0.622597,116.0,65.52,12.07,5.17,29.31,0.00
1,pipeline_3D_prob_0.0_flow_0.2_0_1,0.524975,0.607488,126.0,71.43,9.41,4.76,22.22,1.59
2,pipeline_3D_prob_0.0_flow_0.2_0_2,0.563642,0.567091,131.0,64.89,3.41,4.58,27.48,3.05
3,pipeline_3D_prob_0.0_flow_0.2_0_3,0.512041,0.598560,115.0,72.17,14.97,5.22,22.61,0.00
4,pipeline_3D_prob_0.0_flow_0.2_0_4,0.527047,0.559566,116.0,63.79,10.92,5.17,31.03,0.00
...,...,...,...,...,...,...,...,...,...
1035,pipeline_2D_thr_0.6_flow_0.6_3_3,0.614948,0.615201,193.0,41.45,1.73,39.90,11.40,7.25
1036,pipeline_2D_thr_0.6_flow_0.6_4_0,0.539558,0.567933,126.0,61.11,2.42,7.94,27.78,3.17
1037,pipeline_2D_thr_0.6_flow_0.6_4_1,0.573412,0.552190,127.0,59.06,2.34,8.66,27.56,4.72
1038,pipeline_2D_thr_0.6_flow_0.6_4_2,0.547319,0.599310,168.0,45.83,5.20,30.95,13.69,9.52
